# 4.4.1 Phân cụm khách hàng dựa trên Tổng chi tiêu (Montage) và tần suất mua hàng (Frequency)

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from datetime import datetime


## Tiền xử lý dữ liệu và chuẩn hóa dữ liệu

In [ ]:
# 1. Đọc dữ liệu
df = pd.read_excel("D:/Downloads/KPDL-main/Data/Online Retail.xlsx", engine="openpyxl")

# Loại bỏ các hóa đơn hủy (các giá trị InvoiceNo bắt đầu bằng 'C')
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

# Tính toán cột Total: Số lượng * Đơn giá
df['Total'] = df['Quantity'] * df['UnitPrice']


# 2. Làm sạch dữ liệu

# Loại bỏ các hóa đơn hủy (InvoiceNo bắt đầu bằng 'C')
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]


# Chuyển đổi CustomerID thành kiểu số nguyên để dễ xử lý
df['CustomerID'] = df['CustomerID'].astype(int)

# Tính toán cột "Total" = Số lượng * Đơn giá (Monetary Value cho mỗi giao dịch)
df['Total'] = df['Quantity'] * df['UnitPrice']

# 3. Tổng hợp dữ liệu theo từng khách hàng
customer_agg = df.groupby('CustomerID').agg({
    'InvoiceNo': 'nunique',   # Tần suất giao dịch (số hóa đơn duy nhất) → Frequency
    'Total': 'sum',           # Tổng số tiền chi tiêu của khách hàng → Monetary
    'InvoiceDate': 'max'      # Ngày giao dịch gần nhất → để tính Recency
}).reset_index()  # Reset index để tránh MultiIndex

# Đổi tên cột cho dễ hiểu
customer_agg.rename(columns={'InvoiceNo': 'Frequency', 'Total': 'Monetary'}, inplace=True)

# 4. Tính "Recency" (Số ngày kể từ lần mua gần nhất)
customer_agg['InvoiceDate'] = pd.to_datetime(customer_agg['InvoiceDate'])  # Chuyển về kiểu datetime
reference_date = customer_agg['InvoiceDate'].max()  # Ngày mới nhất trong dữ liệu
customer_agg['Recency'] = (reference_date - customer_agg['InvoiceDate']).dt.days  # Tính số ngày từ lần mua cuối

# 5. Chọn các đặc trưng cần thiết để phân cụm
X = customer_agg[['Monetary', 'Frequency', 'Recency']]

# 6. Chuẩn hóa dữ liệu bằng StandardScaler (giúp mô hình phân cụm hoạt động tốt hơn)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # Chuẩn hóa dữ liệu để đưa về phân phối chuẩn (mean = 0, std = 1)

print(customer_agg.info())
print(customer_agg.head())




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4339 entries, 0 to 4338
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   CustomerID   4339 non-null   int64         
 1   Frequency    4339 non-null   int64         
 2   Monetary     4339 non-null   float64       
 3   InvoiceDate  4339 non-null   datetime64[ns]
 4   Recency      4339 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 169.6 KB
None
   CustomerID  Frequency  Monetary         InvoiceDate  Recency
0       12346          1  77183.60 2011-01-18 10:01:00      325
1       12347          7   4310.00 2011-12-07 15:52:00        1
2       12348          4   1797.24 2011-09-25 13:13:00       74
3       12349          1   1757.55 2011-11-21 09:51:00       18
4       12350          1    334.40 2011-02-02 16:01:00      309
